In [50]:
import glob2
import os
import bs4
import nltk
from transformers import BertTokenizer

In [51]:
os.chdir('/Users/nirmalenduprakash/Documents/Project/NLP/Sentiment Mining')
filenames=glob2.glob('database.mpqa.3.0/gate_anns/**/*.xml')

In [87]:
import re
tokenizer = BertTokenizer.from_pretrained(os.curdir)
seq_train=[]
for file in filenames:
    with open(file,'r') as f:
        soup=bs4.BeautifulSoup(f)
        txt=soup.select('TextWithNodes')[0].text
        sents=getSentences(txt)
        txt_with_nodes=str(soup.select('TextWithNodes')[0])    
        agents=[{'startNode':node['startnode'],\
                       'endNode':node['endnode'], \
                       'text':getNodeText(txt_with_nodes,node['startnode'],node['endnode'])}\
                      for node in soup.select("Annotation[Type=agent]")]
        sentiments=[{'startNode':node['startnode'],\
                       'endNode':node['endnode'], \
                       'intensity':str(list(list(node.children)[3])[3]).split('>')[1].split('<')[0],\
                       'type':str(list(list(node.children)[5])[3]).split('>')[1].split('<')[0]}\
                      for node in soup.select("Annotation[Type=attitude]")]
        targets=[{'startNode':node['startnode'],\
                       'endNode':node['endnode'], \
                       'text':getNodeText(txt_with_nodes,node['startnode'],node['endnode'])}\
                      for node in soup.select("Annotation[Type=sTarget]")]
        for indx,sent in enumerate(sents):
            nodeText=getNodeSent(sent,txt_with_nodes)        
            nodes_in_txt=re.findall('\d+',nodeText)
            sent_agent=None
            sent_target=None
            sent_sentiment_type=None
            sent_sentiment_intensity=None
            
            for node in nodes_in_txt:
                sent_agents=[itm['text'] for itm in agents if itm['startNode']==node]
                if(len(sent_agents)>0):
                    sent_agent=sent_agents[0].replace('  ',' ')
                sent_targets=[itm['text'] for itm in targets if itm['startNode']==node]
                if(len(sent_targets)>0):
                    sent_target=sent_targets[0].replace('  ',' ') 
                sent_intensities=[itm['intensity'] for itm in sentiments if itm['startNode']==node]
                if(len(sent_intensities)>0):
                    sent_sentiment_intensity=sent_intensities[0]
                sent_types=[itm['type'] for itm in sentiments if itm['startNode']==node]
                if(len(sent_types)>0):
                    sent_sentiment_type=sent_types[0]     
#             print('sent:{}'.format(sent),'agent:{}'.format(sent_agent),'target:{}'.format(sent_target),\
#                   'sentiment type:{}'.format(sent_sentiment_type),\
#                   'sentiment intensity:{}'.format(sent_sentiment_intensity))
            
            txt_with_nodes=txt_with_nodes[len(nodeText):]
            labels=tokenizer.tokenize(sent)
            sent=tokenizer.tokenize(sent)
            
            if(sent_agent is not None):
                n=len(tokenizer.tokenize(sent_agent))
                gms=nltk.ngrams(labels,n)
                for i,g in enumerate(gms):
                    if(' '.join(g).strip()==' '.join(tokenizer.tokenize(sent_agent)).strip()):
                        labels[i:i+len(g)]=['B-AG' if j==0 else 'I-AG' for j in range(len(g))]
                        break
                        
            if(sent_target is not None and (sent_agent is not None and sent_agent.strip()!=sent_target.strip())):
                n=len(tokenizer.tokenize(sent_target))
                gms=nltk.ngrams(labels,n)
                for i,g in enumerate(gms):
                    if(' '.join(g).strip()==' '.join(tokenizer.tokenize(sent_target)).strip()):
                        labels[i:i+len(g)]=['B-TG' if j==0 else 'I-TG' for j in range(len(g))]
                        break            
                 
            labels=['O' if w not in ['B-AG','I-AG','B-TG','I-TG'] else w for w in labels]  
            seq_train.append({'sent':sent,'labels':labels})
#             print(seq_train)
#         break

In [52]:
def getNodeText(text,node1,node2):
    text= text[text.index("\""+node1+"\""):text.index("\""+node2+"\"")]
    nodes_in_text=re.findall("\"\d*\"",text)
    for node in nodes_in_text:
        text=text.replace(node,'')    
    text=text.replace('node','')\
              .replace('>','').replace('<','').replace('id','').replace('=','').replace('\"','')\
             .replace('/','')#.replace(node1,'')#.replace(node2,'')

    return text    

In [53]:
def getSentences(text):
    return nltk.sent_tokenize(text)     

In [54]:
def getNodeSent(sent,txt_with_nodes):
#     print(sent,txt_with_nodes)
    result=''
    search_text=txt_with_nodes
    for word in nltk.word_tokenize(sent):  
        try:
            result=result+str(search_text[:search_text.index(word) + len(word)])      
            search_text=txt_with_nodes[len(result):]
        except:
            continue        
    return result 

In [91]:
# getNodeSent(sents[18],str(soup.select('TextWithNodes')[0]))
# txt_with_nodes
sents[14]

'The Sino-US plane collision event that occurred on 1 April can best illustrate the US alternate application of the "two-isms" in its diplomacy.'

In [122]:
lst=[('Following', 'the', 'end'), ('the', 'end', 'of'), ('end', 'of', 'the'), ('of', 'the', 'Cold'), ('the', 'Cold', 'War,'), ('Cold', 'War,', 'the'), ('War,', 'the', 'United'), ('the', 'United', 'States'), ('United', 'States', 'has'), ('States', 'has', 'become'), ('has', 'become', 'the'), ('become', 'the', "world's"), ('the', "world's", 'sole'), ("world's", 'sole', 'superpower'), ('sole', 'superpower', 'and'), ('superpower', 'and', 'the'), ('and', 'the', 'trend'), ('the', 'trend', 'of'), ('trend', 'of', 'pragmatism'), ('of', 'pragmatism', 'in'), ('pragmatism', 'in', 'its'), ('in', 'its', 'foreign'), ('its', 'foreign', 'policy'), ('foreign', 'policy', 'has'), ('policy', 'has', 'become'), ('has', 'become', 'clearer'), ('become', 'clearer', 'and'), ('clearer', 'and', 'clearer.')]

In [77]:
import pandas as pd
df=pd.DataFrame(seq_train)

In [92]:
import pickle
with open('data.pkl','wb') as f:
    pickle.dump(df,f)

In [79]:
import numpy as np
max_len_lbl=np.max([df['labels'].apply(lambda x: len(x))])
max_len_sent=np.max([df['sent'].apply(lambda x: len(x))])
# df['sent']=df['sent'].apply(lambda x:['[CLS]'+x+'[SEP]'])
# df['labels']=df['labels'].apply(lambda x:['[CLS]'+x+'[SEP]'])
max_len=max(max_len_lbl,max_len_sent)

In [88]:
def pad(tokens,max_len,cls=False):
  if(len(tokens)<max_len):
    if(cls):
        return ['[CLS]']+tokens+['[PAD]' for _ in range(max_len-len(tokens)-1)]
    else:
        return tokens+['[PAD]' for _ in range(max_len-len(tokens))]
  else:
    return tokens[:max_len] 

In [89]:
df['labels']=df['labels'].apply(lambda x: pad([l for l in x],max_len))
df['sent']=df['sent'].apply(lambda x: pad([l for l in x],max_len,True))

In [90]:
df['encoding']=df['sent'].apply(lambda x: tokenizer.convert_tokens_to_ids(x))

In [91]:
df['attn_mask']=df['encoding'].apply(lambda x: [0 if tok==str('[PAD]') else 1 for tok in x ])

In [49]:
import pickle 
with open('/Users/nirmalenduprakash/Documents/Project/NLP/classification/val_losses.pkl','rb') as f:
    losses=pickle.load(f)    

In [94]:
len(df.iloc[620]['labels'])

90